In [ ]:
# import MySQL Module
import pymysql
from sqlalchemy import create_engine

# import DataProcessing Module
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import os

engine = create_engine('')
cnn = engine.connect()

## 일별

In [ ]:
'''
2005년~2014년까지의 미세먼지 요인 데이터
'''
data_0514=pd.DataFrame()
for filename in os.listdir(""):
    FD_data=pd.read_csv(''%filename, encoding='utf-8')
    
    # 측정일시 string형식 변환 및 시분단위 제거
    FD_data['측정일시']=FD_data.측정일시.apply(str)
    FD_data['측정일시']=FD_data.측정일시.str[:8]

    # 측정일시 %Y-%m-%d 형식으로 변경
    test_FD_data=pd.to_datetime(FD_data['측정일시'], format="%Y%m%d")
    FD_data['측정일시']=test_FD_data.apply(lambda x: x.strftime('%Y-%m-%d'))

    FD_data[FD_data.columns[0]]=FD_data[FD_data.columns[0]].str[:2]

    # NaN, null값 제거
    for i in range(4,len(FD_data.columns)-1):
            FD_data=FD_data[FD_data[FD_data.columns[i]]>0]

    # 불필요한 칼럼 제거
    del FD_data['주소']
    del FD_data['측정소명']
    del FD_data['측정소코드']
        
    # 지역과 측정일시별로 group by 측정일시의 경우 시간단위로 되어있음
    FD_data=FD_data.groupby([FD_data.columns[0],'측정일시'])['SO2','CO','O3','NO2','PM10'].mean()      
    
    for i in range(0,len(FD_data.columns)):
        FD_data[FD_data.columns[i]]=np.around(FD_data[FD_data.columns[i]],3)
    
    data_0514=data_0514.append(FD_data)

'''
2015년~2016년 1분기까지의 미세먼지 요인 데이터
'''
data_1516= pd.DataFrame()
for filename in os.listdir(""):
    data_2016=pd.read_csv(''%filename, encoding='utf-8')
    data_2016['측정일시']=data_2016.측정일시.apply(str)
    data_2016['측정일시']=data_2016.측정일시.str[:8]

    data_2016[data_2016.PM25 != 99999]
    
    test_data_2016=pd.to_datetime(data_2016['측정일시'], format="%Y%m%d")
    data_2016['측정일시']=test_data_2016.apply(lambda x: x.strftime('%Y-%m-%d'))

    del data_2016['주소']
    del data_2016['측정소명']

    data_2016.columns=['Location','Date','SO2','CO','O3','NO2','PM10','PM25']
    
    data_2016.Location=data_2016.Location.str[0:2]

    for i in range(3,len(data_2016.columns)):
        data_2016=data_2016[data_2016[data_2016.columns[i]]>0]
        
    data_2016=data_2016.groupby(['Location','Date'])['SO2','CO','O3','NO2','PM10','PM25'].mean()

    for i in range(0,len(data_2016.columns)):
        data_2016[data_2016.columns[i]]=np.around(data_2016[data_2016.columns[i]],3)
    
    data_1516=data_1516.append(data_2016)
    
'''
위의 두 데이터프레임을 합쳐서 DB에 넣음
'''
data_0514['PM25']=np.nan
data_0514=data_0514.reset_index()

data_1516=data_1516.reset_index()
data_0514.columns=['Location','Date','SO2','CO','O3','NO2','PM10','PM25']

total=pd.concat([data_0514, data_1516])
total=total.set_index(['Location','Date'])
total.to_pickle('total_dust.pickle')

In [ ]:
'''
기온데이터와 위에 미세먼지 데이터를 합친 데이터프레임 생성
'''
temp_data=pd.read_csv('', encoding='utf-8')
temp_data=temp_data.fillna(0)
unique_data=temp_data[temp_data.columns[0]].unique().tolist()
    
replace_data=['서울','인천','강원','경기','충북','대전','대구','전북','울산','광주','부산','전남','제주','충남','경북','경남']
temp_data[temp_data.columns[0]]=temp_data[temp_data.columns[0]].replace(unique_data,replace_data)
    
temp_data.columns=['Location','Date','Temp','Rainfall','windPower','Humidity','Steam','SeaSteam' \
                  ,'DayLight', 'snow','Cloud','MiddleCloud','LandTemp','FogTimeArr']

data=pd.merge(total.reset_index(),temp_data.reset_index(), on=['Date',temp_data.columns[0]], how='inner').set_index(['Date',temp_data.columns[0]])
data=data.reset_index()
del data['index']

## 월별 미세먼지

In [ ]:
data_monthly = data
data_monthly.Date=data_monthly.Date.str[0:7]
data_monthly=data_monthly.groupby(['Location','Date'])['SO2','CO','O3','NO2','PM10','PM25'].mean()

data_monthly=data_monthly.round(3)
data_monthly=data_monthly.reset_index()

data_totals = data_monthly 
del data_totals['Location']
data_totals=data_totals.groupby(['Date'])['SO2','CO','O3','NO2','PM10','PM25'].mean().round(3)
data_totals['Location']='전국'

In [ ]:
data_totals=data_totals.reset_index()
data_totals=pd.DataFrame(data_totals, columns=['Location','Date','SO2','CO','O3','NO2','PM10','PM25'])

In [ ]:
frame = [data_totals, data_monthly]
result= pd.concat(frame)

In [ ]:
result=pd.DataFrame(result, columns=['Location','Date','SO2','CO','O3','NO2','PM10','PM25'])
result.head()

In [ ]:
result.to_sql('', engine, flavor='mysql',if_exists='replace')

In [ ]:
result.to_pickle('dust_monthly.pickle')

# 월별 미세먼지 + 기상

In [ ]:
data_monthly_air = data
data_monthly_air.Date=data_monthly_air.Date.str[0:7]
data_monthly_air=data_monthly_air.groupby(['Location','Date'])['SO2','CO','O3','NO2','PM10','PM25','Temp','Rainfall','Humidity','Steam','SeaSteam', \
                                                      'DayLight','snow','Cloud','MiddleCloud','LandTemp','FogTimeArr','windPower'].mean()

data_monthly_air=data_monthly_air.round(3)

engine= create_engine('mysql+pymysql://jh:jh@211.180.114.50/armway_db?charset=utf8')
data_monthly_air.to_sql('20161122_month_air_wheather', engine, flavor='mysql',if_exists='append')

# 년별

In [ ]:
data_yearly = data
data_yearly.Date = data_yearly.Date.str[0:4]
data_yearly=data_yearly.groupby(['Location','Date'])['SO2','CO','O3','NO2','PM10','PM25'].mean()

data_yearly=data_yearly.round(3)

In [ ]:
data_yearly=data_yearly.reset_index()
del data_yearly['Location']

In [ ]:
data_yearly_total=data_yearly.groupby(['Date'])['SO2','CO','O3','NO2','PM10','PM25'].mean().round(3).reset_index()
data_yearly_total['Location']='전국'
data_yearly_total.tail()

In [ ]:
data_yearly.tail()

In [ ]:
data_yearly_total=pd.DataFrame(data_yearly_total, columns=['Location','Date','SO2','CO','O3','NO2','PM10','PM25'])

In [ ]:
frames=[data_yearly, data_yearly_total]
result_yearly=pd.concat(frames)

In [ ]:
result_yearly.PM10=result_yearly.PM10.round(2)
result_yearly.PM25=result_yearly.PM25.round(2)

In [ ]:
result_yearly.to_sql('20161122_year_air_quality', engine, flavor='mysql',if_exists='replace')

In [ ]:
data_yearly_air = data
data_yearly_air.Date=data_yearly_air.Date.str[0:4]
data_yearly_air=data_yearly_air.groupby(['Location','Date'])['SO2','CO','O3','NO2','PM10','PM25','Temp','Rainfall','Humidity','Steam','SeaSteam', \
                                                      'DayLight','snow','Cloud','MiddleCloud','LandTemp','FogTimeArr','windPower'].mean()

data_yearly_air=data_yearly_air.round(3)

engine= create_engine('')
data_yearly_air.to_sql('', engine, flavor='mysql',if_exists='append')